<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/english/machine_learning/lessons/keras_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # First try

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path

# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize data set to 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Create a model and add layers
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("cifar-10_model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("cifar-10_model.weights.h5")

Epoch 1/30
782/782 [==============================] - 9s 9ms/step - loss: 1.5504 - accuracy: 0.4336 - val_loss: 1.1824 - val_accuracy: 0.5712
Epoch 2/30
782/782 [==============================] - 7s 9ms/step - loss: 1.1484 - accuracy: 0.5925 - val_loss: 1.0415 - val_accuracy: 0.6276
Epoch 3/30
782/782 [==============================] - 7s 8ms/step - loss: 0.9745 - accuracy: 0.6553 - val_loss: 0.8792 - val_accuracy: 0.6832
Epoch 4/30
782/782 [==============================] - 7s 9ms/step - loss: 0.8724 - accuracy: 0.6939 - val_loss: 0.7765 - val_accuracy: 0.7274
Epoch 5/30
782/782 [==============================] - 7s 8ms/step - loss: 0.7934 - accuracy: 0.7206 - val_loss: 0.7290 - val_accuracy: 0.7469
Epoch 6/30
782/782 [==============================] - 7s 9ms/step - loss: 0.7360 - accuracy: 0.7410 - val_loss: 0.7033 - val_accuracy: 0.7559
Epoch 7/30
782/782 [==============================] - 7s 8ms/step - loss: 0.6945 - accuracy: 0.7582 - val_loss: 0.6722 - val_accuracy: 0.7684
Epoch 

In [ ]:
keras.__version__

'2.15.0'

In [ ]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np

In [ ]:
# Load the json file that contains the model's structure
f = Path("cifar-10_model_structure.json")
my_model_structure = f.read_text()
# Recreate the Keras model object from the json data
my_model = model_from_json(my_model_structure)
# Re-load the model's trained weights
my_model.load_weights("cifar-10_model.weights.h5")


In [ ]:
!wget -q -O dog.jpg https://cdn.britannica.com/79/232779-050-6B0411D7/German-Shepherd-dog-Alsatian.jpg

In [ ]:
img = image.load_img("dog.jpg", target_size=(32, 32))

In [ ]:
list_of_images = np.zeros((1, 32, 32, 3))
image_to_test = image.img_to_array(img)
list_of_images[0] = image_to_test

In [ ]:
class_labels = [
    "airplane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
]

In [ ]:
results = model.predict(list_of_images)
single_result = results[0]
likely_class_indices = np.argsort(single_result)[::-1]
for i in likely_class_indices:
    if single_result[likely_class_indices[i]] > 0.099999:
        print(f"Image is a {class_labels[i]} - Likelihood: {single_result[likely_class_indices[i]]:.2%}")

1/1 [==============================] - 0s 25ms/step
Image is a airplane - Likelihood: 100.00%


# Second try

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
from keras.utils import np_utils
from keras import regularizers
from keras.layers import Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import numpy as np